# 动态图

从飞桨开源框架2.0beta版本开始，飞桨默认为用户开启了动态图模式。在这种模式下，每次执行一个运算，可以立即得到结果（而不是事先定义好网络结构，然后再执行）。

在动态图模式下，您可以更加方便的组织代码，更容易的调试程序，本示例教程将向你介绍飞桨的动态图的使用。


# 设置环境

我们将使用飞桨2.0beta版本，并确认已经开启了动态图模式。

In [6]:
import paddle
import paddle.nn.functional as F
import numpy as np

paddle.disable_static()
print(paddle.__version__)
print(paddle.__git_commit__)


0.0.0
7f2aa2db3c69cb9ebb8bae9e19280e75f964e1d0


# 基本用法

在动态图模式下，您可以直接运行一个飞桨提供的API，它会立刻返回结果到python。不再需要首先创建一个计算图，然后再给定数据去运行。

In [2]:
a = paddle.randn([4, 2])
b = paddle.arange(1, 3, dtype='float32')

print(a.numpy())
print(b.numpy())

c = a + b
print(c.numpy())

d = paddle.matmul(a, b)
print(d.numpy())

[[ 0.40741017  0.2083312 ]
 [-1.7567089   0.72117436]
 [ 0.8870686  -1.1389219 ]
 [ 1.1233491   0.34348443]]
[1. 2.]
[[ 1.4074101   2.208331  ]
 [-0.75670886  2.7211742 ]
 [ 1.8870686   0.86107814]
 [ 2.1233492   2.3434844 ]]
[ 0.8240726  -0.31436014 -1.3907751   1.810318  ]


# 使用python的控制流

动态图模式下，您可以使用python的条件判断和循环，这类控制语句来执行神经网络的计算。（不再需要`cond`, `loop`这类OP)


In [3]:
a = paddle.to_tensor(np.array([1, 2, 3]))
b = paddle.to_tensor(np.array([4, 5, 6]))

for i in range(10):
    r = paddle.rand([1,])
    if r > 0.5:
        c = paddle.pow(a, i) + b
        print("{} +> {}".format(i, c.numpy()))
    else:
        c = paddle.pow(a, i) - b
        print("{} -> {}".format(i, c.numpy()))


0 +> [5 6 7]
1 +> [5 7 9]
2 +> [ 5  9 15]
3 -> [-3  3 21]
4 +> [ 5 21 87]
5 +> [  5  37 249]
6 -> [ -3  59 723]
7 +> [   5  133 2193]
8 -> [  -3  251 6555]
9 -> [   -3   507 19677]


# 构建更加灵活的网络

- 使用动态图可以用来创建更加灵活的网络，比如根据控制流选择不同的分支网络，和方便的构建权重共享的网络。接下来我们来看一个具体的例子，在这个例子中，第二个线性变换只有0.5的可能性会运行。



In [4]:
class MyModel(paddle.nn.Layer):
    def __init__(self, input_size, hidden_size):
        super(MyModel, self).__init__()
        self.linear1 = paddle.nn.Linear(input_size, hidden_size)
        self.linear2 = paddle.nn.Linear(hidden_size, hidden_size)
        self.linear3 = paddle.nn.Linear(hidden_size, 1)

    def forward(self, inputs):
        x = self.linear1(inputs)
        x = F.relu(x)

        if paddle.rand([1,]) > 0.5: 
            x = self.linear2(x)
            x = F.relu(x)

        x = self.linear3(x)
        
        return x     

In [5]:
total_data, batch_size, input_size, hidden_size = 1000, 64, 128, 256

x_data = np.random.randn(total_data, input_size).astype(np.float32)
y_data = np.random.randn(total_data, 1).astype(np.float32)

model = MyModel(input_size, hidden_size)

loss_fn = paddle.nn.MSELoss(reduction='mean')
optimizer = paddle.optimizer.SGD(learning_rate=0.01, 
                                 parameters=model.parameters())

for t in range(200 * (total_data // batch_size)):
    idx = np.random.choice(total_data, batch_size, replace=False)
    x = paddle.to_tensor(x_data[idx,:])
    y = paddle.to_tensor(y_data[idx,:])
    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    if t % 200 == 0:
        print(t, loss.numpy())

    loss.backward()
    optimizer.minimize(loss)
    model.clear_gradients()

0 [2.0915627]
200 [0.67530334]
400 [0.52042854]
600 [0.28010666]
800 [0.09739777]
1000 [0.09307177]
1200 [0.04252927]
1400 [0.03095707]
1600 [0.03022156]
1800 [0.01616007]
2000 [0.01069116]
2200 [0.0055158]
2400 [0.00195092]
2600 [0.00101116]
2800 [0.00192219]


# The end

可以看到使用动态图带来了更灵活易用的方式来组网和训练。